# Pulse Optimization Using GRAPE 

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
This tutorial introduces how to implement the single-qubit gate using the GRAPE (GRadient Ascent Pulse Engineering) algorithm in Quanlse. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Construct Hamiltonian
- Generate optimized pulses using GRAPE
- Summary

## Introduction

**Mechanism**

Gradient-based algorithms are expected to search for the extreme value of an objective function. To search for the maximum value, we can use the gradient ascent algorithm: 
$$
\Theta_n = \Theta_{n-1} + k \bigtriangledown J(\Theta), \tag{1}
$$

The gradient ascent algorithm is analogous to the process of climbing a mountain. If we want to ascent to the peak, we need to know each step's length and direction which give $k\bigtriangledown{J(\Theta)}$ when multiplied. According to the equation above, the position after each step $\Theta_n$ is determined by the product of the derivative of the original position $\Theta_{n-1}$, the objective function $J(\Theta)$ at the original position, and the coefficient $k$. 


The GRAPE algorithm \[1\] is a prototypical method in quantum optimal control - initially introduced on the NMR platform and later extended to other platforms. Experimentally, quantum gates are implemented by external control fields in form of pulse or magnetic flux. By chopping the total control time (for quantum gates) into intervals and assuming the amplitudes of each slice are constant, the amplitudes of the pulses in each slice can be taken as the parameters for optimization. In the Heisenberg picture, we get operator $U (t)$ for dynamics in the period $t \in \left[0, T\right]$：
$$
i\hbar \frac{\partial U (t)}{\partial t} = \hat{H}(t)U(t),
\tag{2}
$$

where $\hbar$ is the reduced Planck constant (here we choose our units so that $\hbar=1$). When the Hamiltonian is time-independent，the solution $U (t = T)$ is
$$
U(T) = \exp(-i\hat{H}T).
\tag{3}
$$

For a time-dependent Hamiltonian, an effective method is to chop the total time $T$ into intervals of the same length, assuming that the Hamiltonian $H_{j}$ is time-independent for each time period $\left[t_j, t_{j+1}\right]$. Here the subscript $j$ denotes the slice sequence. We can solve the respective dynamics $U_{j}$ for each time slice using Eq. (2). The whole process can be represented as:
$$
U = U_{N}U_{N-1}\cdots{U}_{1},
\tag{4}
$$

where $N$ denotes the number of slices.

**Implementation**

Here, we introduce a GRAPE process and set the square of the unnormalized gate fidelity as the objective function $J$:
$$
J = \left|{\rm Tr}\left[(U^{\dagger}_{\rm target}U(T)\right]\right|^2 = \left|{\rm Tr}\left[P^{\dagger}_{j}X_{j}\right]\right|^2,  \ \forall\ 0<j<N \tag{5}
$$

where $U_{\rm target}$ is the target unitary matrix; $X_{j} = U_{j}\cdots{U}_{1}$ is the intermediate propagator; and $P_{j} = U^{\dagger}_{j+1}\cdots{U}^{\dagger}_{N}U_{\rm target}$ is the intermediate back-propagator.  

Continous pulses $u_i(t)$ on the control Hamiltonian $\hat{H}_{i}$ are chopped into discrete functions $u_i(j)$. The partial derivative of the objective function $J$ is given \[2\]:
$$
\dfrac{\partial J}{\partial u_i(j)} = {\rm Re}\{-2 i \Delta t \left[{\rm Tr}(P^{\dagger}_{j}\hat{H}_iX_{j}){\rm Tr}(P^{\dagger}_{j}X_{j})\right]\},
\tag{6}
$$

Gradient-based algorithms will reshape the pulse after each iteration with the learning rate $k$ fixed. Finally, we can obtain pulses with optimized shapes:
$$
u_i(t) \mapsto u_i(t) + k\dfrac{\partial J}{\partial u_i(j)},\tag{7}
$$


![GRAPE](figures/GRAPE1.png)

As shown in the figure above, GRAPE reshapes the pulse's waveform after each iteration.

In the following text, we demonstrate how one can generate optimized pulse sequence for single-qubit gate using GRAPE.

## Preparation

After you have successfully installed Quanlse on your device, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import required packages 
from numpy import dot, round
from Quanlse.Utils.Operator import driveX, driveY, number, duff
from Quanlse.Utils.Tools import project
from Quanlse.Utils import Tools, Plot
from Quanlse.QOperation import FixedGate

# Import GRAPE optimizer
from Quanlse.remoteOptimizer import remoteOptimize1QubitGRAPE

# Import Hamiltonian module
import Quanlse.Utils.Hamiltonian as qham

First, we define constants that describe the overall system, including the sampling period of the arbitrary wave generator (AWG), the system's energy levels to consider, and the gate duration.

In [ ]:
# Sampling period
dt = 0.5

# System energy level
level = 2

# Gate duration (ns)
tg = 40

## Construct Hamiltonian

In this section, we will construct the Hamiltonian. For a two-level system, the Hamiltonian in the rotating frame is made up by：
$$
\hat{H} = \frac{1}{2} \Omega^x(t) (\hat{a}+\hat{a}^{\dagger}) + \frac{1}{2} \Omega^y(t) i(\hat{a}-\hat{a}^{\dagger}),
$$

$\Omega^x(t)$ and $\Omega^y(t)$ are the control pulses on the X channel and Y channel respectively. We construct the Hamiltonian as follows：

In [ ]:
# Initialize the Hamiltonian
ham = qham.createHam(title="1qubit-grape", dt=dt, qubitNum=1, sysLevel=level)

# Add the control terms
qham.addControl(ham, name="q0-ctrlx", onQubits=0, matrices=driveX(level))
qham.addControl(ham, name="q0-ctrly", onQubits=0, matrices=driveY(level))

The optimization usually takes a long time to process on local devices, however, we provide a cloud service that could speed up this process significantly. To use Quanlse Cloud Service, the users need to acquire a token from http://quantum-hub.baidu.com and use the following command to submit the job onto Quanlse's server. For this example, we can write:

In [ ]:
# Import Define class and set the token
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ''

## Generate Optimized Pulses Using GRAPE

Using the function `remoteOptimize1QubitGRAPE()`, we can obtain the optimized pulse on each channel and the infidelity of the unitary evolution. The function `remoteOptimize1QubitGRAPE()` takes in a Hamiltonian dictionary `ham`, a target unitary matrix `ugoal`, the time duration of the gate (`tg`, default at 20 nanoseconds), the maximum iteration (`iterate`, default at 150), and a list consisting the pulse numbers for each channel (`xyzPulses`, default at \[1,1,0\]). The number of slices is determined by $tg / dt$. Here, we set our goal unitary evolution to be the H gate:
$$
U_{\rm target} = 
\dfrac{1}{\sqrt{2}}\begin{bmatrix} 
1 & 1  \\ 1 & -1 
\end{bmatrix} .
$$

In [ ]:
# Define the target unitary evolution
ugoal = FixedGate.H.getMatrix()

# Run the optimization
ham, infid = remoteOptimize1QubitGRAPE(ham, ugoal, tg=tg, iterate=50, xyzPulses=None)

# Print infidelity and plot waves
print(f"minimum infidelity with GRAPE: {infid}")
qham.plotWaves(ham, ["q0-ctrlx", "q0-ctrly"], color = ['blue','mint'], dark=True)

# Print the evolution
result = qham.simulate(ham)["unitary"]
print("The evolution U:\n", round(result, 2))

The quantum gate optimized by the GRAPE algorithm usually differs from the target gate by a global phase $e^{i\phi}$, which will not affect the result.

## Summary

This tutorial introduces how to generate optimized single-qubit gate pulses using the GRAPE algorithm. We can see that GRAPE generate pulses with somewhat irregular shapes. As one of the most common optimization algorithms, GRAPE converges quickly when given a reasonable initial value. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result. (try varying the sampling size when result is undesirable)

The users could follow this link [tutorial-1qubit-grape.ipynb](https://github.com/baidu/Quanlse/tree/master/Tutorial/EN/tutorial-GRAPEoptimizer.ipynb) to the GitHub page of this Jupyter Notebook document to download the code above and further explore the GRAPE algorithm for other gates.

## References

\[1\] [Wilhelm, Frank K., et al. "An introduction into optimal control for quantum technologies." *arXiv preprint arXiv*:2003.10132 (2020).](https://arxiv.org/abs/2003.10132v1)

\[2\] [Khaneja, Navin, et al. "Optimal control of coupled spin dynamics: design of NMR pulse sequences by gradient ascent algorithms." *Journal of magnetic resonance* 172.2 (2005): 296-305.](https://doi.org/10.1016/j.jmr.2004.11.004)